# Numpy: manipulating arrays

Numpy's speedup is no joke.

**Normal Python:**

In [ ]:
import random
data = []
for i in range(1000000):
    data.append(random.gauss(0, 1))

In [ ]:
%%timeit
data2 = []
for x in data:
    data2.append(x**2)

**Numpy:**

In [ ]:
import numpy
data = numpy.random.normal(0, 1, 1000000)

In [ ]:
%%timeit
data2 = data**2

**A Numpy array is everything normal Python data is not:**

   * Loop performed in native bytecode (compiled C)
   * Type-checking performed once before loop
   * Data are packed in contiguous bytes
   * Python's Global Interpreter Lock (GIL) is released during loop

**Bonus:**

   * Most methods also benefit from hardware vectorization

But you have to write your algorithms "sideways."

In [ ]:
px = numpy.random.normal(0, 30, 100000)
py = numpy.random.normal(0, 30, 100000)
pz = numpy.random.normal(0, 300, 100000)

**Computing one event at a time ("normal"):**

In [ ]:
%%timeit
p = numpy.empty(100000)
for i in range(len(p)):                                   # for each px[i], py[i], pz[i]
    p[i] = numpy.sqrt(px[i]**2 + py[i]**2 + pz[i]**2)     # compute p[i]

**Computing one column at a time ("sideways"):**

In [ ]:
%%timeit
p = numpy.sqrt(px**2 + py**2 + pz**2)       # compute all px**2, then all py**2, then all pz**2, then sum all, then sqrt all

Normal math functions are *scalar* (e.g. binary operators like `+` or functions from `import math`). They perform one operation for each appearance in Python source code.

Numpy math functions are *vectorized.* Given equal-length arrays as input, they return the same length array as output, performing all loops in compiled C code, possibly doing 4 or 8 at a time in the CPU.

In [ ]:
small_array = numpy.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [ ]:
small_array**2

In [ ]:
numpy.sqrt(small_array)

In [ ]:
# this won't work because math.sqrt wants a scalar number
import math
math.sqrt(small_array)

Numpy arrays are contiguous blocks of bytes in memory, just like C arrays.

In [ ]:
small_array.view(numpy.uint8)        # view (reinterpret_cast) the 64-bit integers as unsigned 8-bit integers

They can be multidimensional.

In [ ]:
twod = small_array.reshape(2, 5)     # view as 2 arrays of 5 elements each (a constant-time reinterpretation)
twod

They can even be arrays of _structs!_

In [ ]:
table = numpy.array([(0, -9, 0.0), (1, -7, 1.1), (2, -5, 2.2), (3, -3, 3.3), (4, -1, 4.4), (5, 1, 5.5), (6, 3, 6.6), (7, 5, 7.7), (8, 7, 8.8), (9, 9, 9.9)],
                    dtype=[("one", numpy.uint8), ("two", numpy.int64), ("three", numpy.double)])
table

Besides being a common, agreed-upon language for sharing arrays with C/C++ and Fortran code, Numpy has powerful index selection:

In [ ]:
small_array[-1:1:-2]

In [ ]:
twod[1, ::2]

In [ ]:
table["three"][5:]

In [ ]:
table["three"][[False, False, False, False, False, True, False, True, False, True]]

In [ ]:
table["three"][[7, 5, 5, 3, 2, 7]]

In [ ]:
table["three"][table["two"]]

In [ ]:
table["three"][numpy.tile([7, 2], 5)]

The same rules apply to *assigning* to arrays.

In [ ]:
small_array[[False, False, False, False, False, True, False, True, False, True]] = 5000, 7000, 9000

In [ ]:
small_array

Be careful changing things in place!

In [ ]:
view = small_array[5:]
view

In [ ]:
view[0] = 999

In [ ]:
small_array

In [ ]:
view.base is small_array

In [ ]:
small_array.base is None

**Exercise:**

Suppose you're given a zillion `(px, py, pz, E)` 4-vectors and you want `(E, px, py, pz)` 4-vectors. Do it *fast!*

In [ ]:
ZILLION = 1000000
fourvectors = numpy.empty((ZILLION, 4))
fourvectors[:, 0] = numpy.random.normal(0, 1, ZILLION)
fourvectors[:, 1] = numpy.random.normal(0, 1, ZILLION)
fourvectors[:, 2] = numpy.random.normal(0, 10, ZILLION)
fourvectors[:, 3] = numpy.random.normal(0, 10, ZILLION)**2
fourvectors[0]

In [ ]:
%%timeit
reordered = ???

# uproot: ROOT → Numpy

There are several ways to get ROOT data into Numpy arrays:

   * Pythonic iteration in PyROOT (super slow)
   * use ROOT's new `TTree::AsMatrix` (for flat tables)
   * through a custom C++ function
   * with `root_numpy` (compiles against a ROOT version; can segfault with version mismatch)
   * uproot* is a pure-Python, minimal dependency reimplementation of ROOT I/O

<i>(*disclosure: I'm the author)</i>

In [ ]:
!pip install uproot --user

In [ ]:
import uproot

In [ ]:
events = uproot.open("http://scikit-hep.org/uproot/examples/HZZ.root")["events"]
events.keys()

One-per-event ROOT branches become Numpy arrays:

In [ ]:
events.array("MET_px")

Multi-per-event ROOT branches become so-called "jagged arrays":

In [ ]:
events.array("Jet_Px")

A jagged array represents a list (events) of lists (particles).

In [ ]:
px_events, py_events, pz_events = events.arrays(["Jet_Px", "Jet_Py", "Jet_Pz"], outputtype=tuple)

In [ ]:
%%timeit
p = numpy.empty(2773)
j = 0
for i in range(events.numentries):
    for px, py, pz in zip(px_events[i], py_events[i], pz_events[i]):
        p[j] = math.sqrt(px**2 + py**2 + pz**2)
        j += 1

But it consists of three contiguous arrays.

In [ ]:
print(px_events.starts)                # content index where each event starts (inclusive)
print(px_events.stops)                 # content index where each event stops (exclusive)
print(px_events.content)               # content data

In [ ]:
%%timeit
p = numpy.sqrt(px_events.content**2 + py_events.content**2 + pz_events.content**2)

Numpy is designed for flat arrays, so X of all particles → Y of all particles is easy. What about particle attributes *per event?*

In [ ]:
p = numpy.full(events.numentries, numpy.nan)    # initialize with nan
for i in range(events.numentries):
    for px, py, pz in zip(px_events[i], py_events[i], pz_events[i]):
        # only fill first in each event
        if numpy.isnan(p[i]):
            p[i] = math.sqrt(px**2 + py**2 + pz**2)
p

**Step 1:** Which events have at least one jet?

In [ ]:
hasajet = (px_events.stops - px_events.starts != 0)
hasajet

**Step 2:** Get indexes of first jets in events with at least one jet.

In [ ]:
firsts = px_events.starts[hasajet]
firsts

**Step 3:** Gather values from per-particle arrays into per-event arrays.

In [ ]:
px, py, pz = px_events.content[firsts], py_events.content[firsts], pz_events.content[firsts]

**Step 4:** Vectorized calculation, assigned to events with at least one jet.

In [ ]:
p = numpy.full(events.numentries, numpy.nan)       # initialize with nan
p[hasajet] = numpy.sqrt(px**2 + py**2 + pz**2)     # assign to events through the mask
p

**Exercise:**

Compute the invariant mass of the first two muons in each event that has at least two muons.

$m = \sqrt{(E_1 + E_2)^2 - (px_1 + px_2)^2 - (py_1 + py_2)^2 - (pz_1 + pz_2)^2}$

In [ ]:
q_events = events.array("Muon_Charge")
starts, stops = q_events.starts, q_events.stops
energy = events.array("Muon_E").content
px = events.array("Muon_Px").content
py = events.array("Muon_Py").content
pz = events.array("Muon_Pz").content

In [ ]:
mass = ???

In [ ]:
!pip install histbook --user

In [ ]:
from histbook import Hist, bin
# from vega import VegaLite                                             # if Jupyter Notebook, not JupyterLab

In [ ]:
Hist(bin("mass", 200, 0, 200), fill=mass).step(width=400, height=400)   # .to(VegaLite)

![](src/visualization.png)

# Pandas: data analysis on indexed tables

Pandas is a library for manipulating tabular data, but as we've just seen, so is Numpy.

Pandas is based on Numpy. So what's its "value added?"

## Rich indexing!!!

Although the elements of a Numpy array can be accessed in fancy ways, each item is conceptually addressed by a single integer: the entry number. Indexes in Pandas can be typed (e.g. string keys like a `dict`), structured like the two-component index for jagged data below, and sparse (as below).

In [ ]:
df = events.pandas.df(["MET*", "Muon*"])
df

In [ ]:
df.loc[2]

Indexes can be fluidly manipulated. For instance, convert this structured index into integer-indexed columns:

In [ ]:
df.reset_index()

Flatten the sparse subentries into dense columns: a column for each number of muons:

In [ ]:
df2 = df.pivot_table(index="entry", columns="subentry")
df2

In [ ]:
df2[b"Muon_E"]

Numpy concepts and functions apply to Pandas.

In [ ]:
# make a mask of events with at least two muons (second muon index is not nan)
has2muons = numpy.logical_not(numpy.isnan(df2[b"Muon_E", 1]))
df3 = df2[has2muons]

# select particle attributes for the first and second muons through filter
e1,  e2  = df3[b"Muon_E",  0], df3[b"Muon_E",  1]
px1, px2 = df3[b"Muon_Px", 0], df3[b"Muon_Px", 1]
py1, py2 = df3[b"Muon_Py", 0], df3[b"Muon_Py", 1]
pz1, pz2 = df3[b"Muon_Pz", 0], df3[b"Muon_Pz", 1]

# vectorized calculation like Numpy
mass = numpy.sqrt((e1 + e2)**2 - (px1 + px2)**2 - (py1 + py2)**2 - (pz1 + pz2)**2)
mass

In [ ]:
%matplotlib inline
mass.hist()

# Faster math: NumExpr, Numba, and Cython

The Python development strategy is to write slow Python until it gets to be a problem and then accelerate the problem spots.

In [ ]:
# some require actual Numpy arrays, not Pandas series
e1, e2, px1, px2, py1, py2, pz1, pz2 = (
    numpy.array(e1), numpy.array(e2), numpy.array(px1), numpy.array(px2),
    numpy.array(py1), numpy.array(py2), numpy.array(pz1), numpy.array(pz2))

## NumExpr

Mini-compiler to process Numpy arrays in one pass, rather than one per column.

In [ ]:
import numexpr
numexpr.evaluate("sqrt((e1 + e2)**2 - (px1 + px2)**2 - (py1 + py2)**2 - (pz1 + pz2)**2)")

## Numba

More general: uses LLVM to just-in-time compile Python code to native bytecode.

In [ ]:
!pip install numba --user

In [ ]:
import numba

@numba.jit
def f(e1, e2, px1, px2, py1, py2, pz1, pz2):
    out = numpy.empty(len(e1))
    for i in range(len(e1)):
        out[i] = math.sqrt((e1[i] + e2[i])**2 - (px1[i] + px2[i])**2 - (py1[i] + py2[i])**2 - (pz1[i] + pz2[i])**2)
    return out

f(e1, e2, px1, px2, py1, py2, pz1, pz2)

## Cython

Cython is a half-way language between C++ and Python for static compilation and C++ bindings.

In [ ]:
%load_ext Cython

*(continued next page)*

In [ ]:
%%cython -a --cplus -c-std=c++11 -c-O3

import numpy

cdef extern from *:
    """
    void doit(int N, float* out, float* e1, float* e2, float* px1, float* px2,
                                 float* py1, float* py2, float* pz1, float* pz2) {
        for (int i = 0;  i < N;  i++) {
            out[i] = sqrt((e1[i] + e2[i])*(e1[i] + e2[i]) - (px1[i] + px2[i])*(px1[i] + px2[i]) -
                          (py1[i] + py2[i])*(py1[i] + py2[i]) - (pz1[i] + pz2[i])*(pz1[i] + pz2[i]));
        }
    }
    """
    void doit(int N, float* out, float* e1, float* e2, float* px1, float* px2, float* py1, float* py2, float* pz1, float* pz2)

def g(e1, e2, px1, px2, py1, py2, pz1, pz2):
    if (not isinstance(e1, numpy.ndarray) or not isinstance(e2, numpy.ndarray) or
        not isinstance(px1, numpy.ndarray) or not isinstance(px2, numpy.ndarray) or
        not isinstance(py1, numpy.ndarray) or not isinstance(py2, numpy.ndarray) or
        not isinstance(pz1, numpy.ndarray) or not isinstance(pz2, numpy.ndarray)):
        raise TypeError("these are not arrays")

    out = numpy.empty(len(e1), dtype=numpy.float32)    
    doit(len(e1), <float*>(<size_t>out.ctypes.data),
         <float*>(<size_t>e1.ctypes.data), <float*>(<size_t>e2.ctypes.data),
         <float*>(<size_t>px1.ctypes.data), <float*>(<size_t>px2.ctypes.data),
         <float*>(<size_t>py1.ctypes.data), <float*>(<size_t>py2.ctypes.data),
         <float*>(<size_t>pz1.ctypes.data), <float*>(<size_t>pz2.ctypes.data))
    return out

In [ ]:
g(e1, e2, px1, px2, py1, py2, pz1, pz2)

# Dask: delayed and distributed computing

In [ ]:
!pip install "dask[complete]" --user

In [ ]:
lazyarrays = uproot.daskarrays("http://scikit-hep.org/uproot/examples/HZZ.root", "events")

In [ ]:
lazyarrays

In [ ]:
pt = numpy.sqrt(lazyarrays[b"MET_px"]**2 + lazyarrays[b"MET_py"]**2)
pt

`pt` is an *instruction* for a calculation that can be distributed across a cluster. (We don't have a cluster.)

In [ ]:
pt.compute()

# NumPythia, PyJet, and PyPDT: for quick generator-level studies

## NumPythia

In [ ]:
!pip install numpythia --user      # it takes a long time to compile Pythia...

In [ ]:
import numpythia
import numpythia.testcmnd
pythia = numpythia.Pythia(numpythia.testcmnd.get_cmnd("w"), random_state=1)

In [ ]:
import pandas
events = list(pythia(events=1))    # pythia is a generator; evaluate it
pandas.DataFrame(events[0].all())  # event.all(SELECTION) returns a Numpy record array, so of course we Pandas it

## pyjet

In [ ]:
!pip install pyjet --user

In [ ]:
import pyjet

fourvectors = events[0].all()[["E", "px", "py", "pz"]]
for jet in pyjet.cluster(fourvectors, R=1.0, p=-1, ep=True).inclusive_jets():
    print(jet)

## PyPDT

In [ ]:
!pip install pypdt --user

In [ ]:
import pypdt

for pdgid in set(events[0].all()["pdgid"]):
    p = pypdt.get(pdgid)
    if p is not None:
        print("{}: {} mass: {} width {} lifetime {} ctau {}".format(
            pdgid, p.name, p.mass, p.width, p.lifetime, p.ctau))

# The End!